In [ ]:
import os
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA

# --- Carga de la API key (necesaria para el LLM y los embeddings de la pregunta) ---
try:
    with open('../gem_apikey.txt') as f:
        api_key = f.read().strip()
    print("API key cargada correctamente.")
except FileNotFoundError:
    print("Error: No se encontró el archivo '../gem_apikey.txt'.")
    api_key = None
# -----------------------------------------

def iniciar_chat():
    """
    Carga una base de datos vectorial guardada en disco e inicia el chat.
    """
    if not api_key:
        print("Proceso detenido. No se pudo cargar la API key.")
        return

    # --- CONFIGURACIÓN ---
    CARPETA_GUARDADO = "faiss_index_doc1"
    
    # 1. Carga de la base de datos vectorial desde el disco
    print(f"Cargando la base de datos vectorial desde '{CARPETA_GUARDADO}'...")
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=api_key)
    vectorstore = FAISS.load_local(CARPETA_GUARDADO, embeddings, allow_dangerous_deserialization=True)
    
    # 2. Configuración del LLM
    llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.2, google_api_key=api_key)
    
    # 3. Creación de la cadena de RAG
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=vectorstore.as_retriever()
    )
    print("Cadena de RAG lista. ¡El asistente de profesor está listo!")

    print("\n¡El asistente de profesor está listo para responder preguntas sobre el PDF!")
    print("Escribe 'salir' para terminar la conversación.")
    
    # 4. Bucle de chat
    while True:
        query = input("\nPregunta del estudiante: : ")
        if query.lower() == "salir":
            break
        result = qa_chain.invoke({"query": query})
        print("\nRespuesta del asistente:")
        print(result["result"])

if __name__ == "__main__":
    iniciar_chat()

API key cargada correctamente.
Cargando la base de datos vectorial desde 'faiss_index_doc1'...
Cadena de RAG lista. ¡El asistente de profesor está listo!

¡El asistente de profesor está listo para responder preguntas sobre el PDF!
Escribe 'salir' para terminar la conversación.



Pregunta del estudiante: :  que es cardiovascular?



Respuesta del asistente:
El sistema cardiovascular está formado por el corazón y los vasos sanguíneos (arterias, venas y capilares).  Es un sistema de transporte donde el corazón, una bomba muscular, mueve la sangre a través de un circuito cerrado de vasos sanguíneos.  Sus funciones incluyen llevar sustancias a las células para su funcionamiento y eliminar los productos de desecho.



Pregunta del estudiante: :  salir
